In [2]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.utils import get_file
import numpy as np
import argparse
import cv2
import os
import cvlib as cv


Using TensorFlow backend.


In [3]:
model = load_model('weights.best.hdf5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
# open webcam
webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
classes = ['Male','Female']

# loop through frames
while webcam.isOpened():

    # read frame from webcam 
    status, frame = webcam.read()

    if not status:
        print("Could not read frame")
        exit()

    # apply face detection
    face, confidence = cv.detect_face(frame)

    print(face)
    print(confidence)

    # loop through detected faces
    for idx, f in enumerate(face):

        # get corner points of face rectangle        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)

        # crop the detected face region
        face_crop = np.copy(frame[startY:endY,startX:endX])

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue

        # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (178,218))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = model.predict(face_crop)[0]
        print(conf)
        print(classes)

        # get label with max accuracy
        idx = np.argmax(conf)
        label = classes[idx]

        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        Y = startY - 10 if startY - 10 > 10 else startY + 10

        # write label and confidence above face rectangle
        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    # display output
    cv2.imshow("gender detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()


[[352, 273, 460, 403]]
[0.99937505]
[2.4368528e-18 1.0000000e+00]
['Male', 'Female']
[[352, 276, 459, 406]]
[0.999941]
[1.9505411e-05 9.9998045e-01]
['Male', 'Female']
[[352, 277, 458, 406]]
[0.9999546]
[2.4718440e-07 9.9999976e-01]
['Male', 'Female']
[[351, 277, 458, 406]]
[0.9999603]
[4.6454898e-05 9.9995351e-01]
['Male', 'Female']
[[351, 277, 458, 407]]
[0.999959]
[2.1478129e-07 9.9999976e-01]
['Male', 'Female']
[[352, 277, 457, 408]]
[0.9999634]
[5.2761796e-05 9.9994719e-01]
['Male', 'Female']
[[351, 277, 456, 406]]
[0.99997413]
[7.8766237e-07 9.9999917e-01]
['Male', 'Female']
[[351, 278, 455, 407]]
[0.999982]
[4.4519032e-07 9.9999952e-01]
['Male', 'Female']
[[350, 279, 453, 408]]
[0.9999906]
[5.4263573e-06 9.9999452e-01]
['Male', 'Female']
